In [2]:
import os
corpus_root = r'D:\Academics\UF\CIS 4930\Loebner-logs-2003'
folders = os.listdir(corpus_root)
text = []
for folder in folders:
    files = os.listdir(corpus_root+'\\'+folder)
    for file in files:
        f = open(corpus_root+'\\'+folder+'\\'+file, "r")
        for line in f.readlines():
            text.append(line)
        f.close()

In [3]:
text[:50]

['Copyright (C) 2003 Cambridge Center for Behavioral Studies. All rights reserved.\n',
 'ALICE Silver Edition conversing with Judge 05\n',
 '\n',
 '2003-10-18-15-10-28 PROGRAM: Are you there?\n',
 '2003-10-18-15-10-51 JUDGE: yep. what do you think about all this then?\n',
 '2003-10-18-15-10-56 PROGRAM: Do you believe it is better to go it alone? I am Alice, a female Human Homo Sapiens Primate Chrodate Animal.\n',
 "2003-10-18-15-11-26 JUDGE: you didn't answer my question\n",
 '2003-10-18-15-11-31 PROGRAM: OK. Please help me improve my communication skills. What would you have said?\n',
 '2003-10-18-15-11-52 JUDGE: err... never mind. What do you want to talk about now?\n',
 '2003-10-18-15-11-57 PROGRAM: Um. Oh well thanks for trying to explain it to me anyway. Right now? artificial intelligence, robots, art, philosophy, history, geography, politics, and many other subjects.\n',
 '2003-10-18-15-12-22 JUDGE: what do you think about robots?\n',
 "2003-10-18-15-12-27 PROGRAM: Any anthropomo

In [4]:
import nltk
import re
from string import punctuation

In [5]:
#pattern = re.compile(r".*:([a-z|A-Z]+(?:[-='][a-z|A-Z]+)?)+")
pattern = re.compile(r".*: (.*)")
re.findall(pattern, "2003-10-18-16-04-50 JUDGE: i will visit your site when i can")

['i will visit your site when i can']

In [9]:
pattern = re.compile(r".*: (.*)") # for extracting the dialog
pattern2 = re.compile(r"(.*):.*") # for extracting the line info
labeled_sents = []
questions = []
words = set()
for line in text:
    #print(line)
    processed_line = re.findall(pattern,line)
    if len(processed_line) > 0:
        line_info = re.findall(pattern2, line)
        # ignoring judge
        if not re.search('judge', line_info[0], re.IGNORECASE):
            if re.search('confederate', line_info[0], re.IGNORECASE):
                label = 'HUMAN'
                #human_sents.append((processed_line[0], label))
            else:
                label = 'ROBOT'
                #machine_sents.append((processed_line[0], label))
            sents = nltk.sent_tokenize(processed_line[0])
            for sent in list(sents):
                #labeled_sents.append((sent, label))
                if '?' in sent and label == 'HUMAN':
                    questions.append(sent)
                for w in nltk.word_tokenize(sent):
                    if w.isalpha():
                        words.add(w.lower())
            labeled_sents.append((processed_line[0], label))
            

In [10]:
labeled_sents[:10]

[('Are you there?', 'ROBOT'),
 ('Do you believe it is better to go it alone? I am Alice, a female Human Homo Sapiens Primate Chrodate Animal.',
  'ROBOT'),
 ('OK. Please help me improve my communication skills. What would you have said?',
  'ROBOT'),
 ('Um. Oh well thanks for trying to explain it to me anyway. Right now? artificial intelligence, robots, art, philosophy, history, geography, politics, and many other subjects.',
  'ROBOT'),
 ("Any anthropomorphic mechanical being, as those in Karel Capeks play R.U.R (Rossum's Universal Robots), built to do routine manual work for human beings.",
  'ROBOT'),
 ("No I have not seen it. What's it like?", 'ROBOT'),
 ('Interesting comparison.', 'ROBOT'),
 ('What is not to like about it?', 'ROBOT'),
 ('Ah. Better get some rest.', 'ROBOT'),
 ('How many children do you have?', 'ROBOT')]

In [11]:
pronouns = ['i','you','u','they','us','we','them','he','she','him','her','me']
modals = ['can', 'could', 'may', 'might', 'will', 'would', 'shall', 'should', 'must']

In [12]:
#words_fq = nltk.FreqDist(w.lower() for w in words)
#word_features = words_fq.most_common(2000)
#http://aclweb.org/anthology/W15-2517
def features(sent):
    features = {}
    tokenized_sent = [w.lower() for w in nltk.word_tokenize(sent)]
    
    for word in list(words):
        features['words({})'.format(word)] = (word in tokenized_sent)
    #for symbol in list(symbols):
        #features['symbols({})'.format(symbol)] = (tokenized_sent.count(symbol))
    #for symbol in list(symbols):
        #features['symbols({})'.format(symbol)] = (symbol in tokenized_sent)
    # Accuracy: 0.84, 0.87, 0.87, 0.84, 0.84, 0.87, 0.86, 0.82, 0.86, 0.83, 0.84
    # Accuracy with only 50 test cases: 0.88
    # Accuracy also checking existence of symbol: 0.8, 0.86, 0.86
    # Accuracy also checking number of symbol: 0.81, 0.87, 0.83, 0.85
    '''
    for word in list(words):
        features['words({})'.format(word)] = (tokenized_sent.count(word))
    # Accuracy: 0.87, 0.76, 0.81, 0.78, 0.87
    '''
    features['pronouns'] = 0
    for pronoun in pronouns:
        features['pronouns'] += tokenized_sent.count(pronoun)
    features['modals'] = 0
    for modal in modals:
        features['modals'] += tokenized_sent.count(modal)
    #Accuracy with count for words: 0.83, 0.86, 0.83
    #Accuracy with T/F for words: 0.86, 0.87, 0.87, 0.77, 0.82, 0.88, 0.81
    '''
    for pronoun in pronouns:
        features['pronouns'] = (pronoun in tokenized_sent)
        if features['pronouns']:
            break
    for modal in modals:
        features['modals'] = (modal in tokenized_sent)
        if features['modals']:
            break
    '''
    return features

In [13]:
import random
random.shuffle(labeled_sents)
featuresets = [(features(d), c) for (d,c) in labeled_sents]
humanfeatures = [(d,c) for (d,c) in featuresets if c=='HUMAN']
machinefeatures = [(d,c) for (d,c) in featuresets if c=='ROBOT']
#with unbiased test_set, the accuracy is: 0.66, 0.66, 0.73, 0.71, 0.68
#added pronoun and modal presence: 0.72, 0.65, 0.64, 0.65
# added pronoun and modal counts: 0.68, 0.6, 0.68

In [14]:
'''
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
'''

'\nfrom nltk.classify import SklearnClassifier\nfrom sklearn.naive_bayes import BernoulliNB\nfrom sklearn.svm import SVC\n'

In [15]:
#train_set, test_set = featuresets[100:], featuresets[:100]
test_set = humanfeatures[:50] + machinefeatures[50:100]
train_set = featuresets[100:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
#classifier = SklearnClassifier(BernoulliNB()).train(train_set)

In [16]:
print(nltk.classify.accuracy(classifier, test_set))

0.68


In [17]:
classifier.show_most_informative_features(5)

Most Informative Features
            words(shame) = True            HUMAN : ROBOT  =     24.7 : 1.0
        words(afternoon) = True            HUMAN : ROBOT  =     24.7 : 1.0
            words(shall) = True            HUMAN : ROBOT  =     14.8 : 1.0
              words(sit) = True            HUMAN : ROBOT  =     13.7 : 1.0
              words(hot) = True            HUMAN : ROBOT  =     13.7 : 1.0


In [108]:
nltk.sent_tokenize('OK. Please help me improve my communication skills. What would you have said?')

['OK.',
 'Please help me improve my communication skills.',
 'What would you have said?']

In [19]:
question = random.choice(questions)
print (question)
user_ans = input('Your Answer: ')
classifier.classify(features(user_ans))

What shall we talk about?
Your Answer: We shall talk about politics i guess


'ROBOT'